从原始数据集中提取出突变，用annovar进行注释

In [ ]:
import pandas as pd

raw_data_file = "..\data\humu22768-sup-0001-suppmat\humu22768-sup-0002-data.xlsx"
df = pd.read_excel(raw_data_file, index_col=0, sheet_name = None, usecols=["CHR", "Nuc-Pos", "REF-Nuc", "ALT-Nuc"])

In [ ]:
import os

variant_file_dir = "..\data\\variant"
if not os.path.exists(variant_file_dir):
    os.mkdir(variant_file_dir)
for key in df.keys():
        _ = df[key]
        _.to_csv(os.path.join(variant_file_dir, "{}.csv".format(key)))


convert the snp files to vcf

In [ ]:
import pandas 
import os

variant_files_dir = "../data/variant"
output_dir = "../data/variant"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
print("starting")
for root, dirs, files in os.walk(variant_files_dir):
    for file_ in files:
        path = os.path.join(root, file_)
        print("process {}".format(path))
        df_variants = pandas.read_csv(path, index_col=0)
        print(list(df_variants))
        output_file = file_.split(".")[0] + ".vcf"
        if output_file is None:
            output_file = str(uuid.uuid4())
        output = os.path.join(output_dir, output_file)
        with open(output, "w+") as f:
            f.write("##fileformat=VCFv4.3\n")
            f.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n")
            for index, row in df_variants.iterrows():
                chr = index
                pos = row["Nuc-Pos"]
                ref = row["REF-Nuc"]
                alt = row["ALT-Nuc"]
                f.write("{}\t{}\t.\t{}\t{}\t.\t.\t.\n".format(chr, pos, ref, alt))

check the output of annovar

In [ ]:
import pandas as pd

exo_df = pd.read_table("../data/variant/ExoVarFiltered.hg19_multianno.txt").iloc[:,:-11]
# exo_df = exo_df.loc[:,~(exo_df == ".").any()]
cols = list(exo_df)
stats = [[] for _ in range(3)]
for col in cols:
    stats[0].append((exo_df[col]==".").sum())
    stats[1].append((exo_df[col]!=".").sum())
    stats[2].append(col)
print(stats)
import numpy
import matplotlib.pyplot as plt

plt.figure(figsize=(80, 8))
plt.xticks(rotation=45)
size = 5
plt.bar(range(len(cols)), stats[0], label="NaN", tick_label=stats[2])
plt
plt.bar(range(len(cols)), stats[1], label="Non-NaN", bottom = stats[0])
plt.legend()
plt.show()
